# Kuramoto-Shinomoto-Sakaguchi MKV-SDE

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import time
from numpy import linalg as LA
from numpy import mean
from tabulate import tabulate
import mlflow
from itertools import product

from KSS import monte_carlo, base, euler, stochastic_gradient_descent_plot, stochastic_gradient_descent_table, set_experiment, run

## Plots

In [ ]:
def plot_procedure(N, n, r0, rho, M):

    sigma = 0.5
    X0 = 0.5
    M1 = 1000000
    basis_type = 'lagrange'
    eps = 0.01
    h = 0.01
    n = [3, 4, 5, 6, 7, 8]

    T = N * h
    N1 = N
    
    # Euler - Monte Carlo

    start = time.process_time() 
    X, gamma1, gamma2 = monte_carlo(sigma, N1, h, M1, X0)
    end = time.process_time() 

    print("Euler - Monte Carlo execution time: ", end - start)
    print(" ")

    # Stochastic Gradient Descent

    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten() 

    for i in range(6):
        a1_0, a2_0, g = base(N, h, n[i], X0, basis_type)

        start = time.process_time() 
        a1, a2, m = stochastic_gradient_descent_plot(a1_0, a2_0, n[i], r0[i], rho[i], sigma, N, M, X0, eps, h, g, gamma1, gamma2)
        end = time.process_time() 

        print("SGD execution time: ", end - start)
        print("SGD steps: ", m)
        print(" ")

        ax = axes[i] 
        ax.set_xlabel("Time")
        ax.plot(np.linspace(0, T, N+1), np.dot(a1, g), '--', label='$\mathcal{L}\mathbf{a}_m^1$')
        ax.plot(np.linspace(0, T, N+1), gamma1, label='$\overline{\gamma}^{MC}_1$')
        ax.plot(np.linspace(0, T, N+1), np.dot(a2, g), '-.', label='$\mathcal{L}\mathbf{a}_m^2$')
        ax.plot(np.linspace(0, T, N+1), gamma2, label='$\overline{\gamma}^{MC}_2$')
        ax.legend()
        ax.set_title(f"n={n[i]}")

    plt.tight_layout()
    # plt.savefig("KSS_T+str(T)+alln.pdf")
    plt.show()
    return 

### $T = 0.5$

In [ ]:
N = 50
n = [3, 4, 5, 6, 7, 8]
r0 = [5, 5, 5, 5, 5, 5]
rho = [0.7, 0.7, 0.7, 0.9, 0.9, 0.9]
M = 1000

print('T = 0.5')
plot_procedure(N, n, r0, rho, M)

### $T = 1$

In [ ]:
N = 100
n = [3, 4, 5, 6, 7, 8]
r0 = [5, 5, 5, 5, 5, 5]
rho = [0.9, 0.9, 0.8, 0.9, 0.8, 0.9]
M = 1000

print('T = 1')
plot_procedure(N, n, r0, rho, M)

### $T = 2$

In [ ]:
N = 200
n = [3, 4, 5, 6, 7, 8]
r0 = [1, 1, 1, 1, 5, 5]
rho = [0.6, 0.7, 0.6, 0.6, 0.8, 0.7]
M = 1000

print('T = 2')
plot_procedure(N, n, r0, rho, M)

## Tables

In [ ]:
def table_procedure(N, n_list, r0_list, rho_list, M_list):

    sigma = 0.5
    X0 = 0.5
    M1 = 1000000
    eps = 0.01
    h = 0.01
    repetition = 1000
    basis_type = 'lagrange'

    T = N * h
    N1 = N

    # Euler - Monte Carlo

    start = time.process_time()   
    X, gamma1, gamma2 = monte_carlo(sigma, N1, h, M1, X0)
    end = time.process_time()  

    # Stochastic Gradient Descent

    for M in M_list:
        with open("Tables T = "+str(T)+" M= "+str(M)+".txt", "w") as f:

            f.write("Euler - Monte Carlo execution time: "+str(end - start))
            f.write("\n")
            f.write("\n")
            f.write("Number of iterations to achieve convergence with T = "+str(T)+" :")
            f.write("\n")
            f.write("\n")

            m1 = np.zeros((len(n_list), len(M_list)))
            m2 = np.zeros((len(n_list), len(M_list)))
            m3 = np.zeros((len(n_list), len(M_list)))

            for i, n in enumerate(n_list):

                a1_0, a2_0, g = base(N, h, n, X0, basis_type)

                for j, M in enumerate(M_list):
                    
                    r0, rho = r0_list[i][j], rho_list[i][j]

                    start = time.process_time() 
                    mm = [stochastic_gradient_descent_table(a1_0, a2_0, n,r0, rho, sigma, N, M, X0, eps, h, g, gamma1, gamma2) for _ in range(repetition)]
                    end = time.process_time()
                    m1[i, j] = mean(mm)
                    m2[i, j] = min(mm)
                    m3[i, j] = max(mm)

                    f.write(f"Time with n={n}, M={M}, r0={r0} and rho={rho}: {(end - start)/repetition}")
                    f.write("\n")

                print("done n = "+str(n))

            headerslist = [f"M={M}" for M in M_list]
            lateral_headers =[f"n={n}"for n in n_list]

            f.write("\n")
            f.write("\n")

            f.write(f"Average")
            f.write("\n")
            f.write(tabulate(np.column_stack((lateral_headers, m1)), headers=headerslist, colalign=("right",)))
            f.write("\n")
            f.write("\n")
            
            f.write(f"Min")
            f.write("\n")
            f.write(tabulate(np.column_stack((lateral_headers, m2)), headers=headerslist, colalign=("right",)))
            f.write("\n")
            f.write("\n")
            
            f.write(f"Max")
            f.write("\n")
            f.write(tabulate(np.column_stack((lateral_headers, m3)), headers=headerslist, colalign=("right",)))
            f.write("\n")
            f.write("\n")
    return 

#### $T=0.5$

In [ ]:
N = 50
n_list = [3, 4, 5, 6, 7, 8]
M_list = [1, 10, 100, 1000, 10000]

r0_list = [ [5, 5, 5, 5, 5],
            [1, 5, 5, 5, 5],
            [10, 5, 5, 5, 5],
            [10, 5, 5, 5, 5],
            [10, 5, 5, 10, 5],
            [5, 5, 5, 10, 10] ]
rho_list = [ [0.7, 0.7, 0.8, 0.7, 0.9],
                [0.6, 0.6, 0.6, 0.7, 0.9],
                [0.8, 0.8, 0.7, 0.7, 0.6],
                [0.8, 0.7, 0.6, 0.9, 0.6],
                [0.8, 0.6, 0.6, 0.9, 0.6],
                [0.7, 0.6, 0.7, 0.9, 0.8],]

table_procedure(N, n_list, r0_list, rho_list, M_list)

#### $T=1$

In [ ]:
N = 100
n_list = [3, 4, 5, 6, 7, 8]
M_list = [1, 10, 100, 1000, 10000]

r0_list = [ [1, 1, 1, 5, 5],
            [5, 10, 5, 5, 5],
            [1, 5, 5, 5, 5],
            [1, 5, 5, 5, 5],
            [5, 5, 5, 5, 5],
            [1, 5, 5, 5, 5] ]
rho_list = [ [0.6, 0.6, 0.6, 0.9, 0.9],
                [0.9, 0.9, 0.8, 0.9, 0.9],
                [0.6, 0.8, 0.7, 0.8, 0.9],
                [0.6, 0.8, 0.6, 0.9, 0.9],
                [0.8, 0.7, 0.7, 0.8, 0.9],
                [0.6, 0.7, 0.7, 0.8, 0.7],]

table_procedure(N, n_list, r0_list, rho_list, M_list)

#### $T=2$

In [ ]:
N = 200
n_list = [3, 4, 5, 6, 7, 8]
M_list = [1, 10, 100, 1000, 10000]

r0_list = [ [1, 1, 1, 1, 1],
            [1, 1, 1, 1, 1],
            [1, 1, 1, 1, 1],
            [1, 5, 5, 1, 5],
            [1, 1, 5, 5, 5],
            [1, 1, 1, 5, 5] ]
rho_list = [ [0.6, 0.6, 0.6, 0.6, 0.6],
                [0.6, 0.6, 0.6, 0.7, 0.6],
                [0.6, 0.6, 0.6, 0.6, 0.6],
                [0.6, 0.8, 0.8, 0.6, 0.9],
                [0.6, 0.6, 0.8, 0.8, 0.9],
                [0.6, 0.6, 0.6, 0.7, 0.8],]

table_procedure(N, n_list, r0_list, rho_list, M_list)

### Mlflow procedure

In [ ]:
experiment_name = " " 
parent_run_name = " "

shared_params = {
    'sigma':0.5, 'X0':0.5, 'M1':1000000, 'h':0.01, 
    'eps':0.01, 'repetition':1000, 'basis_type':'lagrange',
    }

explored_params = {
    'N':[50, 100, 200],
    'n':[3, 4, 5, 6, 7, 8],
    'r0':[1, 5, 10],
    'rho':[0.6, 0.7, 0.8, 0.9],
    'M': [1, 10, 100, 1000, 10000]
}

def main():

    experiment_id = set_experiment(experiment_name)
    args_list = [] 
    for vals in product(*list(explored_params.values())):
        params_subset = {key:val for key, val in zip(list(explored_params.keys()), vals)}
        params_arg = {**shared_params, **params_subset}
        run_name = '-'.join([f'{k}:{v}' for k, v in params_subset.items()])
        args = [run_name, experiment_name, params_arg]
        args_list.append(args)
    with mlflow.start_run(experiment_id=experiment_id, run_name=parent_run_name):      
        mlflow.log_params(shared_params)
        results = []
        for args in args_list:
            results.append(run(*args))

if __name__ == "__main__":
    main()